
# API Flask com Autenticação Básica e Estrutura com Blueprints

Neste projeto, criamos uma API REST com Flask e MongoDB Atlas organizada em arquivos separados usando **Blueprints**, com **autenticação básica**.

## Pré-requisitos

Instale os pacotes necessários:

```bash
pip install flask pymongo flask-httpauth
```

Crie um cluster no [MongoDB Atlas](https://www.mongodb.com/atlas/database), obtenha sua `CONNECTION_STRING`, e substitua nos exemplos abaixo.



## Estrutura do Projeto (simulada)

```
catalogo_api/
│
├── app.py                # Arquivo principal (executa o app)
├── auth.py               # Autenticação
├── routes/
│   └── livros.py         # Rotas de livros (Blueprint)
├── database.py           # Conexão com MongoDB
```


In [ ]:

# database.py
from pymongo import MongoClient

CONNECTION_STRING = "mongodb+srv://<usuario>:<senha>@<cluster>.mongodb.net/?retryWrites=true&w=majority"

client = MongoClient(CONNECTION_STRING)
db = client['catalogo_livros']
colecao_livros = db['livros']


In [ ]:

# auth.py
from flask_httpauth import HTTPBasicAuth
from flask import jsonify

auth = HTTPBasicAuth()

USUARIOS = {
    "admin": "senha123"
}

@auth.verify_password
def verificar_usuario(usuario, senha):
    if USUARIOS.get(usuario) == senha:
        return usuario
    return None

@auth.error_handler
def erro_autenticacao():
    return jsonify({"erro": "Acesso não autorizado"}), 401


In [ ]:

# routes/livros.py
from flask import Blueprint, request, jsonify
from auth import auth
from database import colecao_livros

livros_bp = Blueprint("livros", __name__)

@livros_bp.route("/livros", methods=["GET"])
@auth.login_required
def listar_livros():
    livros = list(colecao_livros.find({"disponivel": True}, {"_id": 0}))
    return jsonify(livros)

@livros_bp.route("/livros", methods=["POST"])
@auth.login_required
def adicionar_livro():
    dados = request.json
    livro = {
        "titulo": dados["titulo"],
        "autor": dados["autor"],
        "ano": dados["ano"],
        "categorias": dados.get("categorias", []),
        "disponivel": dados.get("disponivel", True)
    }
    colecao_livros.insert_one(livro)
    return jsonify({"mensagem": "Livro adicionado com sucesso!"}), 201


In [ ]:

# app.py
from flask import Flask
from routes.livros import livros_bp

app = Flask(__name__)
app.register_blueprint(livros_bp)

if __name__ == "__main__":
    app.run(debug=True)



## Testes da API com Autenticação

### Listar livros:
```bash
curl -u admin:senha123 http://localhost:5000/livros
```

### Adicionar um novo livro:
```bash
curl -u admin:senha123 -X POST -H "Content-Type: application/json" -d '{
  "titulo": "A Máquina do Tempo",
  "autor": "H.G. Wells",
  "ano": 1895,
  "categorias": ["Ficção Científica"]
}' http://localhost:5000/livros
```

Você também pode testar usando Postman ou Thunder Client no VSCode.
